this notebook is mainly based on [uid42](https://github.com/uid42)'s implementation on [#4901](https://github.com/OpenMined/PySyft/issues/4901). 

This notebook works in 0.4, but dosen't work in 0.3

In [ ]:
import syft as sy
import sklearn
import torch
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt

## Create a virtual machine

In [ ]:
alice = sy.VirtualMachine()
alice_client = alice.get_root_client()

In [ ]:
remote_torch = alice_client.torch

## Send data to alice

In [ ]:
dataset = load_boston()
boston_data = torch.tensor(dataset["data"]).float()
boston_target = torch.tensor(dataset["target"]).float()

boston_data.tag("data")
boston_target.tag("target")

boston_data.send(alice_client, searchable=True)
boston_target.send(alice_client, searchable=True)

ds = [t for t in zip(boston_data, boston_target)]
ds = sy.lib.python.List(ds)
ds.tag("dataset")
ds.send(alice_client, searchable=True)

# check
alice_client.store.pandas

In [ ]:
remote_dl = remote_torch.utils.data.DataLoader(
    alice_client.store[2], batch_size=32, shuffle=True
)
# make the request to be approved automatically(#5015), or exception "Request to access data length not granted"
remote_dl.set_request_config({})

In [ ]:
class Model(sy.Module):
    def __init__(self, torch_ref):
        super(Model, self).__init__(torch_ref=torch_ref)

        self.layer = self.torch_ref.nn.Linear(13, 1)

    def forward(self, x):
        output = self.layer(x)
        return output


local_model = Model(torch)
remote_model = local_model.send(alice_client)

In [ ]:
loss_log = []
for epoch in range(2):
    epoch_loss = 0
    for i, t in enumerate(remote_dl):
        optimizer = remote_torch.optim.Adam(params=remote_model.parameters(), lr=1e-2)
        optimizer.zero_grad()
        data, target = t[0], t[1]
        pred = remote_model(data)
        loss = ((pred.view(-1) - target) ** 2).mean()
        loss.backward()
        optimizer.step()
        gotloss = loss.get()
        print(f"[{epoch} - {i}], loss : {gotloss}")

        epoch_loss += gotloss
    loss_log.append(epoch_loss)

In [ ]:
plt.plot(loss_log)

In [ ]:
remote_model.get().save(f"final_model.pt")
local_model = remote_model.get()

local_loss = 0
with torch.no_grad():
    for i, t in enumerate(remote_dl):
        data, target = t[0], t[1]
        pred = remote_model(data)
        loss = ((pred.view(-1) - target) ** 2).mean()
        gotloss = loss.get()
        local_loss += gotloss
print(f"loss on local model is {local_loss}")